In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import babet as bb
import dask
import cartopy.crs as ccrs
import tqdm
import cartopy.feature as cfeature
from cmcrameri import cm

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
dask.config.set(**{'array.slicing.split_large_chunks': True})

# Import data

In [4]:
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/sfc/pf' # TODO: change as needed
file_dir = {'curr': base_dir.format('curr'),
            'pi': base_dir.format('pi'),
            'incr': base_dir.format('incr')} # TODO: change as needed
babet_exp = {}
for exp in file_dir.keys():
    babet_exp[exp] = xr.open_mfdataset(os.path.join(file_dir[exp], '*.nc'), preprocess=bb.Data.preproc_ds)

In [5]:
# Import ERA5 data
era5_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/ERA5/EU025/sfc/' # TODO: change as needed
era5 = xr.open_mfdataset(os.path.join(era5_dir, '*.nc'))
era5 = era5.sel(time=slice('2023-10-17 00', '2023-10-22 00'))

# Calculate cumulative total precipitation

ERA5: sum up 

IFS: Already cumulative from initialisation, just need to subtract precip at start of window

In [6]:
starttime = '2023-10-19 00' # used to be 18th # Same as Met Office, https://www.metoffice.gov.uk/binaries/content/assets/metofficegovuk/pdf/weather/learn-about/uk-past-events/interesting/2023/2023_08_storm_babet.pdf
endtime = '2023-10-22 00' # Met Office uses 9 hours more

In [7]:
# TODO: adjust method to calculate total precip in 72 hour time period as needed
tp_era = bb.Data.hourly2accum(era5, start_day=starttime, end_day=endtime).tp.sel(time=endtime) # m2mm conversion in function
tp_babet_exp = {key:(value.tp.sel(time=endtime)-value.tp.sel(time=starttime))*1000 for (key,value) in babet_exp.items()}

In [9]:
# Plot settings

# UK
lat_max = 62
lat_min = 47
lon_min = -12
lon_max = 5

# #Europe
# lat_max = 70
# lat_min = 33
# lon_min = -27
# lon_max = 25

tp_min = 0 # Precipitation in mm
tp_max = 200

p_min = 970 # Pressure in hPa
p_max= 1030

euroatlantic = [lon_min-13, lon_max, lat_min-5, lat_max+6]
uk = [-11, 5, 48, 60]

# Comparison between experiments

In [ ]:
# figure and map setup
experiments = ['pi', 'curr', 'incr']
fs = 18
projection = ccrs.PlateCarree()
p_max = 1030
p_min = 970

tp_min = 0
tp_max = 200

fig = plt.figure(1, figsize=(20, 11))
lead_times = ['inidate 2023-10-11', 'inidate 2023-10-13', 'inidate 2023-10-15', 'inidate 2023-10-17'] # TODO: change as needed
inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17'] # TODO: change as needed

# EPS data ------------------    
latitude = babet_exp['pi'].sel(inidate=inidates[0]).latitude.values
longitude = babet_exp['pi'].sel(inidate=inidates[0]).longitude.values

for i, inidate in enumerate(inidates):
    for e, experiment in enumerate(experiments):
        p_vals = (babet_exp[experiment].sel(inidate=inidate, time=slice(starttime, endtime))/100).mean(dim=['number', 'time']).msl.values

        if experiment in ['pi', 'incr']:  # plot difference for counterfactual scenarios
            tp_vals = (tp_babet_exp[experiment].sel(inidate=inidate).mean(dim='number') - tp_babet_exp['curr'].sel(inidate=inidate).mean(dim='number')).values
        else: 
            tp_vals = tp_babet_exp[experiment].sel(inidate=inidate).mean(dim='number').values

        ax = plt.subplot(3,5, i+1+e*5,projection = projection)
        ax.set_extent(uk, projection)
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'grey', zorder = 14)

        # tp as shading
        if experiment =='curr':
            clevs_tp = np.linspace(tp_min, tp_max, 11)  # 17
            cf = ax.contourf(longitude, latitude, tp_vals, clevs_tp, cmap=cm.lapaz_r,
                            transform=projection, zorder = 10, extend = 'max')
        else: 
            clevs_tp = np.linspace(-15, 15, 10)
            cf_diff = ax.contourf(longitude, latitude, tp_vals, clevs_tp, cmap=cm.vik,
                                  transform=projection, zorder = 10, extend = 'both')
        
        # isobars as contours
        clevs_p =np.arange(p_min, p_max, 5)
        cs = ax.contour(longitude, latitude, p_vals, clevs_p, colors='black',
                        transform=projection, zorder = 16)
        plt.clabel(cs, fmt='%d', fontsize=fs-3)
        if experiment=='pi':
            ax.set_title(lead_times[i], size = str(fs))
        else:
            ax.set_title('', size = str(fs))

        # rectangle for Aberdeenshire box
        rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                      edgecolor='k', 
                                      facecolor='none',
                                      transform=projection)
        ax.add_patch(rectangle)
        rectangle.set_zorder(17)

# ERA5 or analysis data ----------------------
latitude = era5.latitude
longitude = era5.longitude

p_vals = era5.msl.sel( time=slice(starttime, endtime)).mean('time').values/100
tp_vals = tp_era.values
ax = plt.subplot(3,5,10,projection = projection)
ax.set_extent(uk, projection)
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'grey', zorder = 14)

# tp as shading
clevs_tp = np.linspace(tp_min, tp_max, 11)  # 17
cf = ax.contourf(longitude, latitude, tp_vals, clevs_tp, cmap=cm.lapaz_r,
                transform=projection, zorder = 10, extend = 'max')

# isobars as contours
clevs_p = np.arange(p_min, p_max, 5)
cs = ax.contour(longitude, latitude, p_vals, clevs_p, colors='black',
                transform=projection, zorder = 16)
plt.clabel(cs, fmt='%d', fontsize=fs-3)
ax.set_title("ERA5", size = fs)

# rectangle for Aberdeenshire box
rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
ax.add_patch(rectangle)
rectangle.set_zorder(17)

# Other figure settings -----------------
ax = plt.subplot(3,5,5)
ax.axis('off')  # removes subplot frame
cax = ax.inset_axes([0.2, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(cf, cax=cax, label='tp (mm)', extend = 'max', shrink=0.8)
cbar.set_label(label='tp (mm)', size=fs)
cbar.ax.tick_params(labelsize=fs-3)

cax_diff = ax.inset_axes([0.6, 0.02, 0.1, 0.95])  # creates inset, [x0,y0, width, height]
cbar_diff = fig.colorbar(cf_diff, cax=cax_diff, label='tp difference (mm)', extend = 'both', shrink=0.8)
cbar_diff.set_label(label='tp difference (mm)', size=fs)
cbar_diff.ax.tick_params(labelsize=fs-3)
plt.figtext(-0.02, 0.82, 'pi', rotation='vertical', size=fs)
plt.figtext(-0.02, 0.48, 'curr', rotation='vertical', size=fs)
plt.figtext(-0.02, 0.15, 'fut', rotation='vertical', size=fs)
plt.suptitle('PGW total precipitation for 18th to 22nd Oct 2023, comparing ensemble mean', size = fs)
plt.tight_layout()

plt.savefig('../figures/A13_tp_all-inidates_all-experiments.png', dpi=300, bbox_inches='tight')
plt.savefig('../figures/A13_tp_all-inidates_all-experiments.pdf')